<a href="https://colab.research.google.com/github/MatthewRomanishin/courses/blob/main/PYDNN/NLP_YELP_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Conv1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras import utils
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np

In [ ]:
KAGGLE_PATH = '/content/gdrive/MyDrive/kaggle/YELP/'
x_train = pd.read_csv(KAGGLE_PATH + 'train.csv')
y_train = pd.read_csv(KAGGLE_PATH + 'train_label.csv', index_col=0)
x_test = pd.read_csv(KAGGLE_PATH + 'test.csv')
x_train.shape, y_train.shape, x_test.shape, 

((560000, 1), (560000, 1), (4000, 1))

In [ ]:
# извлекаем столбец с текстом
x_train = x_train.Review
x_test = x_test.Review


In [ ]:
x_train.nunique()

560000

In [ ]:
# указываем длину словаря, который мы будем использовать
num_words = 100000
# указываем длину последовательности, к которой приведем все тексты
max_review_len = 100
# создаем и обучаем токенизатор на тернировочных данных
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(x_train)
# применяем токенизатор к тренировочным и тестовым данным
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
# приводим все вектора текстов к общей длине
x_train = pad_sequences(x_train, maxlen=max_review_len, padding='post')
x_test = pad_sequences(x_test, maxlen=max_review_len, padding='post')

In [ ]:
model = Sequential()
model.add(Embedding(num_words, 128, input_length=max_review_len))
model.add(GRU(64, return_sequences=True))
model.add(GRU(32,return_sequences=True))
model.add(GRU(64, return_sequences=True))
model.add(GRU(16,return_sequences=True))
model.add(GRU(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(32, activation='tanh'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,
                    y_train, 
                    epochs=3,
                    batch_size=1024,
                    validation_split=0.1)

Epoch 1/3
493/493 [==============================] - 105s 213ms/step - loss: 0.3043 - accuracy: 0.8640 - val_loss: 0.1701 - val_accuracy: 0.9363
Epoch 2/3
493/493 [==============================] - 103s 209ms/step - loss: 0.1666 - accuracy: 0.9368 - val_loss: 0.1379 - val_accuracy: 0.9479
Epoch 3/3
493/493 [==============================] - 104s 211ms/step - loss: 0.1279 - accuracy: 0.9533 - val_loss: 0.1396 - val_accuracy: 0.9485


In [ ]:
best_score = max(history.history['accuracy'])
best_score

0.9767618775367737

In [ ]:
sample_submission = pd.read_csv(KAGGLE_PATH + 'sample_submission.csv', index_col='id')
sample_submission.label = model.predict_classes(x_test)
sample_submission.to_csv("sample_submission.csv")

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
